In [2]:
import pandas as pd
import json
import networkx as nx

In [3]:
df = pd.read_excel('Travel.xlsx')
df.columns = df.iloc[0]
df.drop(0, inplace = True)
Arcos = []
for index, row in df.iterrows():
    Arcos.append([row[2], row[3], row[7], row[8], row[9]])

In [4]:
LocStart = [Arcos[i][0] for i in range(0,len(Arcos))]    #Origen arco
LocEnd = [Arcos[i][1] for i in range(0,len(Arcos))]      #Destino arco
TimeEmpty = [Arcos[i][2] for i in range(0,len(Arcos))]   #Tiempo vacío arco
TimeLoad = [Arcos[i][3] for i in range(0,len(Arcos))]    #Tiemp cargado arco
Closed = [Arcos[i][4] for i in range(0,len(Arcos))]      #Camino cerrado (1 o 0

#Crear grafo completo desde "tabla" MTV
Nodos = pd.unique(LocStart+LocEnd) 
Grafo = nx.DiGraph()
Grafo.add_nodes_from(Nodos)

for i in range(0,len(Arcos)):
    Grafo.add_edge(LocStart[i], LocEnd[i], time_empty = TimeEmpty[i], time_load = TimeLoad[i], closed = Closed[i])

# Ejecutar algoritmo Floyd_Warshall
predecessors, distance = nx.floyd_warshall_predecessor_and_distance(Grafo, weight='time_empty')
predecessors_loaded, distance_loaded = nx.floyd_warshall_predecessor_and_distance(Grafo, weight='time_load')
#Se construye diccionario (NodoInicio,NodoTérmino)->(lista camino minimo, Lista tiempos acumulados)
caminosMinimosYTiemposAcumulados = {}




In [5]:
for nodoInicio in Nodos:
    for nodoTermino in Nodos:
        camino = nx.reconstruct_path(nodoInicio,nodoTermino,predecessors)
        tiempos = []
        tiempoAcumulado = 0
        tiempos.append(tiempoAcumulado)
        for i in range(0,len(camino)-1):
            tiempoAcumulado += distance[camino[i]][camino[i+1]]
            tiempos.append(tiempoAcumulado)
        caminosMinimosYTiemposAcumulados[(nodoInicio,nodoTermino)] = (camino,tiempos)

In [6]:
MTV = {}
for o in distance.keys():
    MTV[o] = {}
    for d in distance[o].keys():
        MTV[o][d] = {}
        if o == d:
            MTV[o][d][0] = 0
            MTV[o][d][1] = 0
            
        else:
            try:
                MTV[o][d][0] = distance[o][d]/60 
                MTV[o][d][1] = distance_loaded[o][d]/60
                
            except:#Si el grafo no es conexo relleno
                print((o, d))

In [7]:
with open('MTV.txt', 'w') as file:
     file.write(json.dumps(MTV)) # use `json.loads` to do the reverse

In [8]:
with open("MTV.txt") as tweetfile:
    MTV = json.loads(tweetfile.read())